In [27]:
%matplotlib inline

import numpy as np
import itertools

import torch
import torch.nn as nn
import torch.optim as optim

import random
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook

In [28]:
# set random seed to 0
np.random.seed(0)
torch.manual_seed(0)
torch.set_default_tensor_type('torch.DoubleTensor')

In [71]:
def vectorize(s, vocab_size=5):
    vector = np.zeros((len(s)*vocab_size,), dtype=np.float64)
    for i in range(len(s)):
        c = s[i]
        if c != '0':
            vector[i*vocab_size + int(c) - 1] = 1
    return vector

def vectorize_2d(s, vocab_size):
    vectors = np.zeros((len(s), vocab_size), dtype=np.float64)
    for i in range(len(s)):
        c = s[i]
        if c != '0':
            vectors[i][int(c) - 1] = 1
    return vectors

In [84]:
def fill_missing_number(s):
    """
    Missing number: Given a string of n unique digits in [0, n], one of which is 0,
        fill in the place of 0 the missing digit from [1, n]
    """
    input_set = set(s)
    new_s = str(s)
    assert '0' in input_set
    assert len(input_set) == len(s) # all unique numbers
    assert not [1 for c in s if int(c) > len(s)] # No number greater than len(s)
    diff = {str(i) for i in range(1, len(s)+1)} - set(s)
    return new_s.replace('0', diff.pop())

def shift(s, n):
    """
    Shift the position of each digit by n, wrapping around
    """
    assert n < len(s)
    return s[n:] + s[:n]
    
def add_n(s, n):
    """
    Add n to each digit in s, mod len(s)
    """
    
def reverse(s):
    return s[::-1]

def clear_nth_digit(s):
    """
    The first digit of s determines which digit to clear
    """
    index = int(s[0])
    assert index < len(s)
    new_s = s[:index] + '0' + s[index+1:]
    return new_s

def pointer_task(s):
    output = ""
    index = 1
    for i in range(len(s)):
        index = s[int(index)-1]
        output += index
    return output

def vectorized_pointer_task(s):
    indices = np.zeros((len(s), len(s)))
    
    index = 0
    for i in range(len(s)):
        index = int(s[index]) - 1
        indices[i][index] = 1
    
    return indices

In [63]:
vectorized_pointer_task('01232')

array([[0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.]])

In [64]:
def create_data(prefix_chars, suffix_chars, prefix_length, suffix_length):
    prefixes = list(itertools.product(prefix_chars, repeat=prefix_length))
    suffixes = list(itertools.product(suffix_chars, repeat=suffix_length))
    return [''.join(p + s) for p, s in itertools.product(prefixes, suffixes)]
    
create_data('345', '123', 3, 3)

['333111',
 '333112',
 '333113',
 '333121',
 '333122',
 '333123',
 '333131',
 '333132',
 '333133',
 '333211',
 '333212',
 '333213',
 '333221',
 '333222',
 '333223',
 '333231',
 '333232',
 '333233',
 '333311',
 '333312',
 '333313',
 '333321',
 '333322',
 '333323',
 '333331',
 '333332',
 '333333',
 '334111',
 '334112',
 '334113',
 '334121',
 '334122',
 '334123',
 '334131',
 '334132',
 '334133',
 '334211',
 '334212',
 '334213',
 '334221',
 '334222',
 '334223',
 '334231',
 '334232',
 '334233',
 '334311',
 '334312',
 '334313',
 '334321',
 '334322',
 '334323',
 '334331',
 '334332',
 '334333',
 '335111',
 '335112',
 '335113',
 '335121',
 '335122',
 '335123',
 '335131',
 '335132',
 '335133',
 '335211',
 '335212',
 '335213',
 '335221',
 '335222',
 '335223',
 '335231',
 '335232',
 '335233',
 '335311',
 '335312',
 '335313',
 '335321',
 '335322',
 '335323',
 '335331',
 '335332',
 '335333',
 '343111',
 '343112',
 '343113',
 '343121',
 '343122',
 '343123',
 '343131',
 '343132',
 '343133',
 '343211',

In [66]:
for s in create_data('456', '123', 3, 3):
    print(s, vectorized_pointer_task(s))

444111 [[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
444112 [[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
444113 [[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
444121 [[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
444122 [[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
444123 [[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
444131 [[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
444132 [[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1.

In [80]:
vectorize_2d("545231", 6)

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [85]:
pointer_task("545231")

'535353'

In [86]:
vectorize_2d(pointer_task("545231"), 6)

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.]])

In [88]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(1, 51, dtype=torch.double)
        c_t = torch.zeros(1, 51, dtype=torch.double)
        h_t2 = torch.zeros(1, 51, dtype=torch.double)
        c_t2 = torch.zeros(1, 51, dtype=torch.double)
        
        for input_t in input.chunk(input.size(1), dim=1):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

'1423'

In [30]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128

In [92]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(x_tensor, y_tensor):
    hidden = rnn.initHidden()
    rnn.zero_grad()

    x_tensor_eos = addEOS(x_tensor)
    x_tensor = zeroPad(x_tensor, 2)
    for i in range(x_tensor_eos.size()[0]-1): # Start outputing predictions at EOS
        output, hidden = rnn(x_tensor_eos[i], hidden)
        
    out_str = ""
    for i in range(y_tensor.size()[0]):
        output, hidden = rnn(zeroPad(output, 1), hidden)
        loss = criterion(output, torch.max(y_tensor[i], 1)[1])
        loss.backward(retain_graph=True)
        out_char = i2l[int(torch.max(output, 1)[1][0])]
        out_str += out_char

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return out_str, loss.item()

[('5', '6'), ('6', '5')]

[(('1', '2', '3', '4'), ('5', '6')),
 (('1', '2', '3', '4'), ('6', '5')),
 (('1', '2', '4', '3'), ('5', '6')),
 (('1', '2', '4', '3'), ('6', '5')),
 (('1', '3', '2', '4'), ('5', '6')),
 (('1', '3', '2', '4'), ('6', '5')),
 (('1', '3', '4', '2'), ('5', '6')),
 (('1', '3', '4', '2'), ('6', '5')),
 (('1', '4', '2', '3'), ('5', '6')),
 (('1', '4', '2', '3'), ('6', '5')),
 (('1', '4', '3', '2'), ('5', '6')),
 (('1', '4', '3', '2'), ('6', '5')),
 (('2', '1', '3', '4'), ('5', '6')),
 (('2', '1', '3', '4'), ('6', '5')),
 (('2', '1', '4', '3'), ('5', '6')),
 (('2', '1', '4', '3'), ('6', '5')),
 (('2', '3', '1', '4'), ('5', '6')),
 (('2', '3', '1', '4'), ('6', '5')),
 (('2', '3', '4', '1'), ('5', '6')),
 (('2', '3', '4', '1'), ('6', '5')),
 (('2', '4', '1', '3'), ('5', '6')),
 (('2', '4', '1', '3'), ('6', '5')),
 (('2', '4', '3', '1'), ('5', '6')),
 (('2', '4', '3', '1'), ('6', '5')),
 (('3', '1', '2', '4'), ('5', '6')),
 (('3', '1', '2', '4'), ('6', '5')),
 (('3', '1', '4', '2'), ('5', '6')),
 

In [103]:
list(itertools.product('abcd', repeat=3))

[('a', 'a', 'a'),
 ('a', 'a', 'b'),
 ('a', 'a', 'c'),
 ('a', 'a', 'd'),
 ('a', 'b', 'a'),
 ('a', 'b', 'b'),
 ('a', 'b', 'c'),
 ('a', 'b', 'd'),
 ('a', 'c', 'a'),
 ('a', 'c', 'b'),
 ('a', 'c', 'c'),
 ('a', 'c', 'd'),
 ('a', 'd', 'a'),
 ('a', 'd', 'b'),
 ('a', 'd', 'c'),
 ('a', 'd', 'd'),
 ('b', 'a', 'a'),
 ('b', 'a', 'b'),
 ('b', 'a', 'c'),
 ('b', 'a', 'd'),
 ('b', 'b', 'a'),
 ('b', 'b', 'b'),
 ('b', 'b', 'c'),
 ('b', 'b', 'd'),
 ('b', 'c', 'a'),
 ('b', 'c', 'b'),
 ('b', 'c', 'c'),
 ('b', 'c', 'd'),
 ('b', 'd', 'a'),
 ('b', 'd', 'b'),
 ('b', 'd', 'c'),
 ('b', 'd', 'd'),
 ('c', 'a', 'a'),
 ('c', 'a', 'b'),
 ('c', 'a', 'c'),
 ('c', 'a', 'd'),
 ('c', 'b', 'a'),
 ('c', 'b', 'b'),
 ('c', 'b', 'c'),
 ('c', 'b', 'd'),
 ('c', 'c', 'a'),
 ('c', 'c', 'b'),
 ('c', 'c', 'c'),
 ('c', 'c', 'd'),
 ('c', 'd', 'a'),
 ('c', 'd', 'b'),
 ('c', 'd', 'c'),
 ('c', 'd', 'd'),
 ('d', 'a', 'a'),
 ('d', 'a', 'b'),
 ('d', 'a', 'c'),
 ('d', 'a', 'd'),
 ('d', 'b', 'a'),
 ('d', 'b', 'b'),
 ('d', 'b', 'c'),
 ('d', 'b'